In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
import os
os.chdir('C:\\Users\\naviy\\OneDrive\\Documents\\CHRIST UNIVERSITY, BANGALORE')

In [3]:
data = pd.read_csv("C:\\Users\\naviy\\OneDrive\\Documents\\CHRIST UNIVERSITY, BANGALORE\\NSSO68.csv")

C:\Users\naviy\AppData\Local\Temp\ipykernel_13092\2840125202.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\naviy\\OneDrive\\Documents\\CHRIST UNIVERSITY, BANGALORE\\NSSO68.csv")


In [4]:
data['state_1'].unique()

array(['GUJ', 'ORI', 'CHTSD', 'MP', 'JRKD', 'WB', 'AP', 'MH', 'D&D',
       'D&NH', 'MIZ', 'TRPR', 'MANPR', 'ASSM', 'MEG', 'NAG', 'A&N',
       'PNDCRY', 'TN', 'GOA', 'KA', 'KE', 'LKSDP', 'SKM', 'Bhr', 'UP',
       'RJ', 'ARP', 'DL', 'HR', 'Pun', 'HP', 'UT', 'Chandr', 'J$K'],
      dtype=object)

In [5]:
#Subsetting the data
subset_data = data[data['state_1'] == 'AP'][['foodtotal_q', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data)

       foodtotal_q  MPCE_MRP  MPCE_URP  Age  Meals_At_Home  \
6777     18.308732   3818.86   3780.50   35           60.0   
6778     29.781670   4100.08   4322.00   24           60.0   
6779     18.412530   2333.55   1769.40   45           60.0   
6780     24.025527   2284.85   1986.25   36           60.0   
6781     22.070518   1952.06   1224.20   36           84.0   
...            ...       ...       ...  ...            ...   
79719    17.800203   1304.09   1344.50   26           90.0   
79720    47.825207   2581.20   2613.50   50           60.0   
79721    13.250154   1374.35   1820.50   32           60.0   
79722    21.725427   1391.12   1900.50   18           90.0   
79723     0.000000   1536.62   1348.00   14            0.0   

       Possess_ration_card  Education  No_of_Meals_per_day  
6777                   1.0        7.0                  2.0  
6778                   2.0       10.0                  2.0  
6779                   1.0        8.0                  2.0  
6780       

In [6]:
#Checking for missing values
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Age'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(data['Education'].isna().sum())

0
0
0
0
7


In [7]:
#Creating a function to impute th emissing values with the mean of the variable
def impute_with_mean(data, columns):
    for column in columns:
        data[column].fillna(data[column].mean(), inplace=True)
    return data

In [8]:
#Imputiong the columns
columns_to_impute = ['Education', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'foodtotal_q']

In [9]:
subset_data = impute_with_mean(subset_data, columns_to_impute)

In [10]:
print(subset_data.isna().sum()) 

foodtotal_q            0
MPCE_MRP               0
MPCE_URP               0
Age                    0
Meals_At_Home          0
Possess_ration_card    0
Education              0
No_of_Meals_per_day    0
dtype: int64


In [11]:
#Fitting the regression model
X = subset_data[['MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = subset_data['foodtotal_q']


In [12]:
model = sm.OLS(y, X).fit()

In [13]:
#Printinf the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            foodtotal_q   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     515.8
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:23:34   Log-Likelihood:                -23675.
No. Observations:                6899   AIC:                         4.736e+04
Df Residuals:                    6892   BIC:                         4.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   8.3477    

In [14]:
#Checking for multicollinearity using Inflator Factor (VIF)
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]


In [15]:
print(vif_data)

               feature        VIF
0                const  61.884058
1             MPCE_MRP   2.566446
2             MPCE_URP   2.311461
3                  Age   1.134833
4        Meals_At_Home   1.146450
5  Possess_ration_card   1.213771
6            Education   1.374596


In [16]:
#Extracting the coefficients from the model
coefficients = model.params

In [17]:
#Constructing the equation
equation = f"y = {coefficients[0]:.2f}"
for i in range(1, len(coefficients)):
    equation += f" + {coefficients[i]:.6f}*x{i}"
print(equation)


y = 8.35 + 0.001962*x1 + 0.000718*x2 + 0.150340*x3 + 0.081055*x4 + -3.097385*x5 + 0.039711*x6
